# Putting together my library

I have several thousand books in 3 main formats and I'd like to combine them all so I can get a better picture of the books that I own. I also share an Audible account with my cousin and would like to combine her Audible library with my own. Then we can both use the library to make sure we don't buy books we already have. 

I have some books in more than one format, but I'd like to merge them all and say which formats I have instead of multple listings of the same book in different formats. At the moment, I have all my ebooks cataloged in Calibre. I can add my audiobooks (mp3, audible, etc) to those entries but I'd have to do it one-by-one, which I'd like to avoid. I have a library website running in docker in a Synology home server, but it uses the database from Calibre to populate the books. 

I decided that putting all my books together in one csv file with only one entry per book, regardless of the format, I could import them directly into Calibre as a separate library and use that for the web. Eventually, I'd like to add more information so I can do some statistics on the files, but I'm not up to that challenge...yet.

## Import all the things

In [1]:
import numpy as np
import pandas as pd

import datetime

#import itertools

In [2]:
audible = pd.read_csv('my_data/audible.csv')
audible2 = pd.read_csv('my_data/second_audible.csv')
abooks = pd.read_csv('my_data/audiobooks.csv')
ebooks = pd.read_csv('my_data/ebooks.csv')
goodreads = pd.read_csv('my_data/goodreads.csv')

**Dataframes and what they contain**
1. audible (audible.csv) - contains all the audio books in my audible library (extracted using ALE browser extension)
2. audible2 (second_audible.csv) - contains all the audio books in my cousin's audible library (extracted using ALE)
3. abooks (audiobooks.csv) - contains all audiobooks I own in mp3 format - shouldn't be in audible
4. ebooks (ebooks.csv) - contains most of my ebooks, but is missing some from Amazon
5. goodreads (goodreads.csv) - contains all the books from my goodreads shelves

All of these files should be fairly clean, but some are more complete than others. I need to get the same information for them all, which may require scraping the data from amazon or goodreads. Eventually I want all the data in one file and have at least these columns:
- Title
- Author(s)
- Tag
- Comments (blurb)
- Page numbers or length of audiofile
- Series (won't apply to all)
- ISBN or other id number

When they're imported to Calibre, I can add covers easily. Eventually, I'd like to include images in the data so I can use a nicer search function, but for now I'm keeping it as simple as possible. I'd also like to add finished reading dates and genres so I can play around with statistics.

## My ebooks

I'll start off getting my ebook dataframe organized. I'm starting with this one because this is the format I'll need to use to import everythinng into Calibre later. This is a list of all the ebooks that I have in Calibre already. I still have more to add from Amazon.

In [3]:
ebooks.head()

,title,authors,series,series_index,#series,#series_index,comments,tags,timestamp,isbn,identifiers,#pages,pubdate,publisher
0,BF03-A Dragon's Ascension,Ed Greenwood,Band of Four,3.0,NaN,NaN,Aglirta is known as the Kingless Land. Once a ...,"Fantasy, Dragon, Epic",2019-06-11T17:14:09-04:00,9780765341440,"isbn:9780765341440,google:mhgx03oeVJUC",397.0,2003-02-17T00:00:00-05:00,Macmillan
1,A Fiend in Need,Maryjanice Davidson,Queen Betsy,4.5,NaN,NaN,"""A Fiend in Need"" reveals the true identity of...","Short Story, Humorous, Paranormal, Romance, Sh...",2019-06-11T17:06:06-04:00,9780425207956,"amazon:B000PDYVUC,isbn:9780425207956,google:QF...",59.0,2006-04-04T00:00:00-04:00,Berkley
2,"The Blades of the Rose Bundle: Warrior, Scound...",Zoe Archer,NaN,1.0,NaN,NaN,****Zoe Archer’s BLADES OF THE ROSE series—fea...,"Fiction, Historical, Romance, Stempunk, Suspen...",2019-01-23T23:33:35-05:00,9781420122954,"isbn:9781420122954,google:zOZIM73fXIwC,mobi-as...",1912.0,2010-11-30T23:54:00-05:00,Zebra
3,BR03-Rebel,Zoë Archer,NaN,1.0,NaN,NaN,"On the Canadian frontier in 1875, nature is a ...","Paranormal, Romance, Historical",2019-06-11T17:32:44-04:00,NaN,NaN,465.0,2010-10-31T20:00:00-04:00,Zebra
4,BR04-Stranger,Zoë Archer,NaN,1.0,NaN,NaN,He protects the world’s magic -- with his scie...,paranormal romance,2019-06-11T17:32:43-04:00,9781420119862,isbn:9781420119862,529.0,2010-11-30T19:00:00-05:00,NaN


In [4]:
ebooks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5922 entries, 0 to 5921
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5922 non-null   object 
 1   authors        5922 non-null   object 
 2   series         2066 non-null   object 
 3   series_index   5922 non-null   float64
 4   #series        4 non-null      object 
 5   #series_index  4 non-null      float64
 6   comments       5147 non-null   object 
 7   tags           4949 non-null   object 
 8   timestamp      5922 non-null   object 
 9   isbn           4140 non-null   object 
 10  identifiers    5618 non-null   object 
 11  #pages         4212 non-null   float64
 12  pubdate        5922 non-null   object 
 13  publisher      4956 non-null   object 
dtypes: float64(3), object(11)
memory usage: 647.8+ KB


### Preliminary Thoughts

This data is pretty clean already since it's from Calibre. I want to drop the identifier column since it's the same as the ISBN column. I'll drop the hashes in front of column names. I want to keep only the year for the pubdate and only the mm/dd/yyyy from the timestamp. I'm only keep that one so I'll be able to add new ebooks more easily later. There are some titles with a code in front that I'll want to drop. This helped keep series organized on older Kindles, but now they get put into collections so it's not needed anymore.

In [5]:
# get a list of the column names
ebooks.columns

Index(['title', 'authors', 'series', 'series_index', '#series',
       '#series_index', 'comments', 'tags', 'timestamp', 'isbn', 'identifiers',
       '#pages', 'pubdate', 'publisher'],
      dtype='object')

In [6]:
# drop the columns I know I don't want
ebooks = ebooks[['title', 'authors', 'series', 'series_index',
                 '#series', '#series_index', 'comments', 'tags', 
                 'timestamp', 'isbn', '#pages', 'pubdate', 'publisher']]

In [7]:
# rename the columns
columns = ['title', 'authors', 'series', 'series_index',
           'series2', 'series2_index', 'comments', 'tags', 
           'timestamp', 'isbn', 'pages', 'pubdate', 'publisher']
ebooks.columns = columns

In [14]:
ebooks.head()

,title,authors,series,series_index,series2,series2_index,comments,tags,timestamp,isbn,pages,pubdate,publisher
0,A Dragon's Ascension,Ed Greenwood,Band of Four,3.0,NaN,NaN,Aglirta is known as the Kingless Land. Once a ...,"Fantasy, Dragon, Epic",2019-06-11,9780765341440,397.0,2003,Macmillan
1,A Fiend in Need,Maryjanice Davidson,Queen Betsy,4.5,NaN,NaN,"""A Fiend in Need"" reveals the true identity of...","Short Story, Humorous, Paranormal, Romance, Sh...",2019-06-11,9780425207956,59.0,2006,Berkley
2,"The Blades of the Rose Bundle: Warrior, Scound...",Zoe Archer,NaN,1.0,NaN,NaN,****Zoe Archer’s BLADES OF THE ROSE series—fea...,"Fiction, Historical, Romance, Stempunk, Suspen...",2019-01-23,9781420122954,1912.0,2010,Zebra
3,Rebel,Zoë Archer,NaN,1.0,NaN,NaN,"On the Canadian frontier in 1875, nature is a ...","Paranormal, Romance, Historical",2019-06-11,NaN,465.0,2010,Zebra
4,Stranger,Zoë Archer,NaN,1.0,NaN,NaN,He protects the world’s magic -- with his scie...,paranormal romance,2019-06-11,9781420119862,529.0,2010,NaN


### Start Cleaning

I want to:
- drop my code from the beginning of any titles that still have it
- change timestamp column to datetime and only keep the year/month/day
- only keep the year in the pubdate column - it doesn't need to be datetime
- put brackets around the series number in both series number columns

In [9]:
# get rid of code at beginning of some titles
# if a book has a hyphen, then split the column at the hyphen and use only the second half of the title (after the code)
drop_code = ebooks['title'].str.split('-').str[1]
# otherwise, keep the title as it is
ebooks['title'] = np.where(ebooks['title'].str.contains('-'), drop_code, ebooks['title'])

In [10]:
# get rid of hours/min/sec by dropping everything after the T
ebooks['timestamp'] = ebooks['timestamp'].str.split('T').str[0]

In [11]:
# there were some rows with 100 as the year, so I changed those to 1/1/2019
# any date in the past would work since I'm only going to use this to pull out the recent books later
ebooks.loc[ebooks['timestamp'].str.contains('100'), 'timestamp'] = '2019-01-01'

In [12]:
# convert the column to datetime
# the pubdate still has some issues to work on
ebooks['timestamp']= pd.to_datetime(ebooks['timestamp'], infer_datetime_format = True)

In [13]:
# keep just the year for this column
ebooks['pubdate'] = ebooks['pubdate'].str.split('-').str[0]

In [ ]:
ebooks.info()

## My Audible

In [ ]:
audible.head()

In [ ]:
audible.info()

In [ ]:
audible.isnull().sum()

### Preliminary Thoughts

Most of the columns are fine, but I'll need to change the book_numbers to a float (there are books like 2.5), then enclose them all in brackets before sending to Calibre. A few narrators and lengths are missing, so I'll need to look those up and add them. Several tags and categories are missing, but I may be able to whittle those down when I combine this dataframe with ebooks. Then again, it might make things even harder. I'll have to take a look at the ebooks to see. A few of the columns (store page changed, and isbns) have no values, so I'll drop those right away. Here's the plan for the columns:

|     | Column                                | Plan                                                                                      |
|:--: |:--------------------------------------|:-----------------------------------------------------------------------------------------|
| [X] | Added                                 | Drop                                                                                      |
| [X] | Title, Title Short                    | Drop (will use Title Short)                                                              |
| [X] | Series                                | Keep only series name                                                                    |
| [X] | Book Numbers                          | Change to series_number, then add brackets around numbers                                |
| [ ] | Blurb                                 | Compare to Summary                                                                        |
| [X] | Authors                               | Doesn't need changes, but take a look at unique values just in case                      |
| [ ] | Narrators                             | Fill in missing values                                                                    |
| [X] | Tags                                  | delete any NaN values, see if I can fill missing with Categories                          |
| [X] | Categories                            | Can I use this to fill in Tags column - More trouble than it's worth - drop              |
| [X] | Parent & Child Category               | Is this the same as the Category column? YES - Drop                                      |
| [X] | Length                                | put in any missing information - info won't be found in Calibre                          |
| [X] | Progress                              | Drop                                                                                      |
| [X] | Release Date                          | Drop                                                                                      |
| [X] | Publisher                             | Drop                                                                                      |
| [X] | My Rating, Rating & Ratings           | Drop                                                                                      |
| [X] | Favorite                              | Drop                                                                                      |
| [X] | Format                                | Drop                                                                                      |
| [X] | Language                              | Drop                                                                                      |
| [X] | Whispersync                           | Drop                                                                                      |
| [X] | From Plus Catalog                     | Do I own these or are they only in Plus? I don't own them - Drop                          |
| [X] | Unavailable                           | Do I need this? - NOPE - dropped it                                                      |
| [X] | Archived & Downloaded                 | Drop                                                                                      |
| [X] | Store Page Changed                    | Drop                                                                                      |
| [X] | Store Page Missing                    | Do I need this? - Nope - dropped it                                                      |
| [X] | ASIN, ISBN, and ISBN13                | Keep only ASIN                                                                            |
| [ ] | Summary                               | Compare with blurb                                                                        |
| [X] | People Also Bought                    | Drop                                                                                      |
| [X] | Store Page URL                        | Keep as audible_url and add a column in Calibre for it                                    |
| [X] | Sample                                | Drop                                                                                      |
| [X] | Web Player                            | Drop                                                                                      |
| [X] | Cover                                 | Change to cover_url and add a column in Calibre for it                                    |
| [X] | Search in Goodreads                   | Drop if goodreads book id isn't included, else extract id                                |

### Start Cleaning

In [ ]:
# get a list of column names to change to lower case and subset
cols = audible.columns
cols

In [ ]:
# convert all columns to lower case and replace spaces with _
cols = [x.lower().replace(' ', '_') for x in cols]

# assign new column names to dataframe
audible.columns = cols

### Drop columns

Drop these columns:
Added, Title, Progress, Release Date, Publishers, My Rating, Rating, Ratings, Favorite, Format, Language, Whispersync, Archived, Downloaded, Store Page Changed, ISBN 10, ISBN 13, People Also Bought, Sample, Web Player

In [ ]:
# drop columns listed in above table by subsetting df
audible = audible[['title_short', 'series', 'book_numbers', 'blurb', 
                   'authors', 'narrators', 'tags', 'categories', 
                   'parent_category', 'child_category','length',
                   'from_plus_catalog', 'unavailable', 'store_page_missing', 
                   'asin', 'summary', 'store_page_url', 'cover', 
                   'search_in_goodreads']]

### Change column names

These column names will be changed:
title_short (title), book_numbers (series_number), store_page_url (audible_url), cover (cover_url)

In [ ]:
audible.rename(columns={'title_short':'title', 'book_numbers':'series_number', 'store_page_url': 'audible_url', 'cover':'cover_url'}, inplace=True)

### Examine columns (quick)

These just need a quick check before deciding to drop them or keep them:  

- categories (keep for now)
- parent_category (drop)
- child category (drop)
- from_plus_catalog (drop rows, then drop column)
- unavailable (drop)
- store_page_missing (drop)

#### categories, parent_category, child_category

In [ ]:
cat = audible[['tags', 'categories', 'parent_category', 'child_category']]
cat.head(2)

Since parent & child category columns have the same information as categories, I'll drop those two. Since categories and tags has the same information, I'll keep categories until I narrow everything else down. Then I'll decide if there are enough missing tag values that I can fill with categories to make it worth cleaning up categories and moving them to tags.

In [ ]:
audible.drop(['parent_category', 'child_category'], axis = 1, inplace = True)

#### from_plus_catalog

In [ ]:
audible['from_plus_catalog'].unique()

In [ ]:
audible[audible['from_plus_catalog'] == True]

These are books that I've borrowed from Plus or Kindle Unlimited. I don't own them, so I'll drop the rows, and then drop the column.

In [ ]:
# drop rows that list Plus/KU books
audible.drop(audible[audible['from_plus_catalog'] == True].index, inplace = True)
audible.reset_index(inplace = True)

In [ ]:
# see how many rows are left after dropping books from Plus/KU
audible.shape[0]

There were 29 non-null values in the from_plus_catalog column. 1505-29 = 1476, so it dropped the rows correctly. Now to drop the column.

In [ ]:
audible.drop('from_plus_catalog', axis = 1, inplace = True)

#### unavailable

In [ ]:
audible['unavailable'].unique()

It looks like my 1 unavailable book was in the plus catalog. I'll go ahead and drop that column since it's only null now.

In [ ]:
audible.drop('unavailable', axis = 1, inplace = True)

#### store_page_missing

In [ ]:
audible['store_page_missing'].unique()

In [ ]:
audible[audible['store_page_missing'] == True].head()

I'm not sure what store_page_missing is supposed to mean, but I thought it meant there was no url. There aren't any missing urls, however, so I'll drop store_page_missing.

In [ ]:
audible.drop('store_page_missing', axis = 1, inplace = True)

#### search_in_goodreads

In [ ]:
# check for the goodreads book id in this column
# if no id, drop the column
audible.iloc[1]['search_in_goodreads']

The search in goodreads column is literally a search in goodreads for the author and title. It doesn't have the book id, so I can drop that column.

In [ ]:
# drop columns
audible.drop('search_in_goodreads', axis = 1, inplace = True)

### Columns that are going to take more work

The columns need worked on or need more investigating to decide what to do with them:

 - series (drop (book #) in all rows)
 - series_number
 - narrator
 - tags (change NaN values to blank)
 - categories (only 9 more than tags, so drop)
 - blurb
 - summary

In [ ]:
# let me take a look at what still needs done
audible.info()

#### categories

I was going to copy the categories into the tags column, but that will only fix 9 of them. Since I'd have to do a bit of work on the categories before copying it, it's easier to drop the categories column and let Calibre fix the tags later.

In [ ]:
# drop categories column 
audible.drop('categories', axis = 1, inplace = True)

#### tags
All NaN values need replaced with a blank. There are & symbols between a few words, as well as the comma. I want to get rid of those and convert the words into a list in each cell.

In [ ]:
# change Nan to blank
audible.tags = audible.tags.fillna('')

In [ ]:
# first change the & to a comma
audible.tags = audible.tags.str.replace(' & ', ', ')

In [ ]:
# this will split the words in the tags column at the comma
# each cell in the column will be a list of words
audible.tags = audible.tags.str.split(', ')

In [ ]:
#take a look at some of the new column values
audible.iloc[0:20]['tags']

#### series

There are NaN values in series which will need to be replaced with blanks. The non-null entries have the series and then in parenthesis, the book numbers. Since I already have a column with book numbers, I need to get rid of the parenthesis and what's in them.

In [ ]:
audible['series'] = audible['series'].str.replace(r"\(.*\)","", regex = True)
audible.head()

#### series_number

There should be brackets around every entry. Some entries are a range of numbers, so I'll need to change those, too. I'll change the ranges to 0 so it won't interfere with any actual numbers that aren't ranges.

In [ ]:
# first I need to change any ranges to 0 so the column will be integers instead of objects.


In [ ]:
# then add brackets around numbers

In [ ]:
# then change Nan to blank (don't want empty brackets so this is last)
audible.series_number = audible.series_number.fillna('')

In [ ]:
#examine columns more closely - I'll group some of them so I can compare them separately
blurb = audible[['blurb', 'summary']]

In [ ]:
print('Blurb\n', blurb.loc[0, 'blurb'])
print('\nSummary\n', blurb.loc[0,'summary'])

Summary is the best description, but has HTML tags. The summary column also has some NaNs, while the blurb columns doesn't. I'll fill the missing summaries from the blurbs and get rid of the HTML in the summaries that I have.